# Subword Tokenization

In this exercise, we will learn how to train our own subword tokenizers with different algorithms: BPE and Unigram. We will use `sentencepiece`, a library from Google to help create our tokenizers.

## Ref:
https://github.com/google/sentencepiece/blob/master/python

## Setup

In [1]:
# !wget https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt
# !wget https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/kratoo-40000000-40002000.jsonl

## Code

In [2]:
import sentencepiece as spm
import io
import json

Load data

In [3]:
pantip_text = []
with open('kratoo-40000000-40002000.jsonl', 'r') as json_file:
    json_list = list(json_file)
    for json_str in json_list:
        result = json.loads(json_str)
        pantip_text.append(f"{result['title']}\n{result['content']}\n")
sum([len(t) for t in pantip_text])

1060318

In [4]:
with open("pra-apai-manee-ch1-50.txt") as f:
  pra_apai_manee_data = f.readlines()

In [5]:
sum([len(t) for t in pra_apai_manee_data])

1100605

In [6]:
pantip_train_text = pantip_text[:int(len(pantip_text)*0.8)]
pantip_test_text = pantip_text[int(len(pantip_text)*0.8):]

pam_train_text = pra_apai_manee_data[:int(len(pra_apai_manee_data)*0.8)] #pam = pra_apai_manee
pam_test_text = pra_apai_manee_data[int(len(pra_apai_manee_data)*0.8):]

## Run tokenizer training

The Python wrapper provides multiple APIs for training our tokenizers

1. `spm.SentencePieceTrainer.train(input='input.txt', model_prefix='m', vocab_size=vocab_size, model_type=model_type)`
  <br> This will output the tokenizer files `m.model` and `m.vocab` that can be later loaded into `SentencePieceProcessor`.
  <br><br>
2. `spm.SentencePieceTrainer.train(sentence_iterator=iterator, model_writer=obj_with_write_method, vocab_size=vocab_size, model_type=model_type)`
  <br> This method will require a file object e.g. `obj_with_write_method = io.BytesIO()`. The advantage of this method is you can run sentencepiece on environments that have limited access to the local file system. But you will still have to save the model file if you want to re-use the model else you will have to train it again.
<br><br>
3.  `spm.SentencePieceTrainer.train('--input=input.txt --model_prefix=m --vocab_size=vocab_size --model_type=model_type')`
<br> Same as no.1




### Unigram tokenizer

We are going to start with training a unigram tokenizer. You can use any method of training one. Make sure to set vocab_size to 1000.

In [12]:
## Train
model_pam_uni = io.BytesIO()
spm.SentencePieceTrainer.train(sentence_iterator=iter(pam_train_text), model_writer=model_pam_uni, vocab_size=1000)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: 
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0

### Q1 MCV

How many tokens did you get when tokenizing the following sentence with your unigram tokenizer: <br>
'อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม'

In [13]:
sp_pam_uni = spm.SentencePieceProcessor(model_proto=model_pam_uni.getvalue())

In [14]:
len(sp_pam_uni.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str))

29

### BPE Tokenizer

Now try training a BPE tokenizer.

In [21]:
model_pam_bpe = io.BytesIO()
spm.SentencePieceTrainer.train(sentence_iterator=iter(pam_train_text), model_writer=model_pam_bpe, vocab_size=1000, model_type='bpe')

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: 
  model_type: BPE
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0
  d

### Q2 MCV

How many tokens did you get when tokenizing the following sentence with your BPE tokenizer: <br>
'อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม'

In [22]:
sp_pam_bpe = spm.SentencePieceProcessor(model_proto=model_pam_bpe.getvalue())
print(len(sp_pam_bpe.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)))

28


These are some of your vocabs. Note that you will see "▁" (U+2581) in every type of tokenizer in SentencePiece since it makes it possible to perform detokenization \(unsplit your sentences\) without relying on language-specific resources.

In [24]:
unigram_vocabs = [sp_pam_uni.id_to_piece(id) for id in range(sp_pam_uni.get_piece_size())]
" | ".join(unigram_vocabs[:500])

'<unk> | <s> | </s> | ▁ | า | เ | น | ม | ย | ก | ร | ว | ด | ส | ง | บ | ค | มา | อ | ล | จะ | ท | ให้ | ห | ไป | ไม่ | แ | ว่า | พ | ุ | ี | ๏ | ฯ | ข | ช | เป็น | พระ | โ | ที่ | ใจ | ▁จะ | จ | ะ | ิ | ต | ก็ | อยู่ | ป | ได้ | ่ | ไ | เข้า | ู | ▁พระ | ้า | ตาม | ใน | ้ | ▁แล้ว | เหมือน | รา | ศ | เจ้า | เห็น | ลา | กัน | ั | หา | นาง | ทรง | ประ | ์ | ยา | ัก | ํา | ซ | าน | ัง | ฉ | องค์ | ัด | แล้ว | อน | ดู | ถ | ด้วย | มี | ▁จึง | นี้ | ่า | ผ | น้อง | แต่ | ทํา | ▁นาง | ▁ให้ | รัก | พี่ | คิด | ลูก | พา | รู้ | การ | กับ | ัน | หน้า | กระ | วน | ออก | ่อ | เขา | ถึง | ระ | ข้า | ับ | พล | นั่ง | ทั้ง | หน | รับ | ษ | กล | วง | ลง | ฝ | กร | พร | ความ | เสีย | ดี | ขึ้น | อง | ่ง | ธ | ▁แต่ | คน | กลับ | ▁ฝ่าย | ้น | อด | ภ | หรือ | ตร | ือ | ฟัง | แม่ | ▁ไม่ | ไว้ | ยัง | ▁เห็น | นา | ขอ | มิ | น้ํา | หล | ดัง | ▁พอ | ▁ทั้ง | ช่วย | สม | นั้น | ริ | ทัพ | ต้อง | วัน | อา | น้อย | รบ | ิน | อย่า | เอา | จน | เรา | สุด | เสียง | ข้าง | หลัง | ตี | ตัว | ละ | สุ | วัง | ทุก | ่น

In [23]:
bpe_vocabs = [sp_pam_bpe.id_to_piece(id) for id in range(sp_pam_bpe.get_piece_size())]
" | ".join(bpe_vocabs[:500])

'<unk> | <s> | </s> | ้า | ่า | อง | ระ | ํา | รา | อย | ่ง | มา | จะ | ัง | ัน | ▁เ | าย | ้ว | ับ | ี่ | ม่ | อน | ให | าม | ้น | ็น | พระ | ีย | าง | กล | ้ง | ัก | หน | ให้ | ไม่ | หล | ่น | ึง | ▁แ | ทั | ตร | าร | ้อง | ไป | ิด | ข้า | ว่า | หม | คร | ือ | ล้ว | เป | เส | ประ | าน | ั่ง | ▁๏ | ▁ฯ | ที่ | อก | เล | ิน | ได | พล | ทร | ัด | นาง | ึก | ได้ | ู่ | ▁จะ | ค์ | ี้ | พร | เป็น | สุ | ทั้ง | อม | ัย | เร | ห็น | ▁จ | ▁พระ | ก็ | ใจ | อา | ื่ | ่าง | ต่ | กร | ิง | วง | วน | ือน | เจ | ู้ | ียง | อยู่ | รร | ตาม | ▁พ | ้วย | าว | ถึง | คล | ั้น | รี | เข | ด้วย | สม | องค์ | สน | าก | ▁แล้ว | เช | ัว | ย์ | ใน | คว | น้ | หมือน | ▁ส | ูก | อบ | กระ | เจ้า | ทรง | ลา | กัน | มี | ่าย | พรา | ิ่ง | เข้า | เห็น | ิต | สง | อด | ณ์ | วย | ้ม | คิด | เม | เก | เด | ▁นาง | วา | ุก | ▁ให้ | ดู | หา | ▁อ | ▁จึง | ทํา | ลง | รัก | เค | แล้ว | ่าน | พี่ | เหมือน | ั่น | ความ | ยง | อย่า | หร | มิ | ืน | ช่ | การ | ัญ | ▁ไม่ | ฝ่าย | ศรี | ้าง | วก | ้อม | ือง | น้อง | ยว | พา | แก |

### User-defined symbols

Another important concept to know of is User-defined symbols. These special symbols are reserved for a special purpose \(e.g.\, the \<MASK\> token used in BERT) and will always be tokenized into one token.

Refer to the documentation for ways to add these special tokens to your tokenizer.

https://github.com/google/sentencepiece/blob/master/python

## Train another tokenizer on another domain

Now try training another unigram tokenizer on `pantip_text` and we will use it to compare with the unigram tokenizer we trained earlier.

In [26]:
## Train
# spm.SentencePieceTrainer.train(input='./kratoo-40000000-40002000.jsonl', model_prefix='pantip_unigram', vocab_size=1000)
model_pantip_uni = io.BytesIO()
spm.SentencePieceTrainer.train(sentence_iterator=iter(pantip_train_text), model_writer=model_pantip_uni, vocab_size=1000)

sp_pantip_uni = spm.SentencePieceProcessor(model_proto=model_pantip_uni.getvalue())
print(len(sp_pantip_uni.encode('อรุณสวัสดิ์ ฉันเอามเหสีมาหาม สวัสดี ประเทศไทยสบายดีไหม', out_type=str)))

27


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: 
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0

## Analyse top tokens on different datasets

Use your tokenizers to tokenize the datasets and analyse your most common vocabularies (try 300-400 vocabs with len>1). Hint: tokenize your data and count the tokens.

In [30]:
word_count_pam = {}
for line in pam_test_text:
    for word in sp_pam_uni.encode(line, out_type=str):
        word_count_pam[word] = word_count_pam.get(word, 0) + 1

for i in word_count_pam:
    if len(i) > 1 and word_count_pam[i] > 300:
        print(i, word_count_pam[i])


ว่า 520
พระ 462
จะ 622
มา 666
ที่ 417
ให้ 650
ไป 657
อยู่ 314
ไม่ 536
ยา 321
▁จะ 421
เป็น 387
ใจ 313
รา 326


In [31]:
word_count_pantip = {}
for line in pantip_test_text:
    for word in sp_pantip_uni.encode(line, out_type=str):
        word_count_pantip[word] = word_count_pantip.get(word, 0) + 1

for i in word_count_pantip:
    if len(i) > 1 and word_count_pantip[i] > 300:
        print(i, word_count_pantip[i])

ของ 375
จะ 677
ได้ 579
เป็น 525
ที่ 1145
นี้ 329
ค่ะ 318
ไป 654
ให้ 405
้า 411
แล้ว 366
เรา 576
มี 536
ก็ 535
ว่า 555
มา 651
ไม่ 475
เลย 375
ใน 436
การ 445


### To answer
What are some notable differences you see between the two vocabs?

Write your answer below.

**ANS.**

คำที่ปรากฏบ่อยในพระอภัยมณีเป็นคำที่ใช้แสดงความรู้สึกบอกว่าใครทำอะไรและเล่าเรื่อง เช่น ใจ อยู่ ไป ที่ เป็น

ส่วนคำใน pantip จะเป็นคำที่ใช่เร่าเรื่องราวในชีวิตประจำวันเช่น เรา แล้ว ก็ ใน ที่ เลย การ

## Using tokenizer across domains

One problem you may face is your dataset is very specialized. In that case the tokenizer trained on a general domain may not perform as good as it should when used on your dataset.

Next you will try using tokenizers trained on one general domain (on Pantip) and use it on a specialized domain (พระอภัยมณี) and vice versa.

### Q3 MCV

What percentage increase do you observe when tokenizing the whole พระอภัยมณี dataset with a tokenizer trained on Pantip compared to the one trained on พระอภัยมณี.

In [41]:
train_pam_test_pam_count = 0
train_pantip_test_pam_count = 0

for line in pra_apai_manee_data:
    train_pam_test_pam_count += len(sp_pam_uni.encode(line, out_type=str))
    train_pantip_test_pam_count += len(sp_pantip_uni.encode(line, out_type=str))

print((train_pantip_test_pam_count - train_pam_test_pam_count) / train_pam_test_pam_count * 100, "%")

43.42710745209495 %


### Q4 MCV

What percentage increase do you observe when tokenizing the whole Pantip dataset with a tokenizer trained on พระอภัยมณี compared to the one trained on Pantip.

In [37]:
train_pantip_test_pantip_count = 0
train_pam_test_pantip_count = 0

for line in pantip_text:
    train_pantip_test_pantip_count += len(sp_pantip_uni.encode(line, out_type=str))
    train_pam_test_pantip_count += len(sp_pam_uni.encode(line, out_type=str))

print((train_pam_test_pantip_count - train_pantip_test_pantip_count) / train_pantip_test_pantip_count * 100, "%")

12.855248847916101 %


### To answer
Why do you think the number of tokens tokenized by the general tokenizer (the one trained on Pantip) has a higher percentage increase compared to the number of tokens tokenized by the specialized tokenizer? (Hint: we fixed vocab size.)

**ANS.**

เพราะเมื่อเรา fix ขนาด vocab, model จะเลือกเรียนคำที่ปรากฎบ่อยซึ่ง คำในพระอภัยมณีแม้จะเป็นกลอนแต่หลายๆคำน่าจะ เป็นคำที่เจอทั่วไปในชิวิตประจำวัน(เราเลยพออ่านกลอนรู้เรื่องบ้าง) ทำให้เอาไปใช้กับ pantip แล้วมีคำที่รู้จักเยอะ ส่วนคำที่กลอนชอบใช้แต่ไม่เจอบ่อยก็จะกินที่ vocab และทำให้เกิด error ขึ้น (~13%)

แต่ model ที่ถูก train ด้วย pantip จะเรียนแต่คำที่เจอในชีวิตประจำวันและไม่ค่อยเจอคำที่ใช้เขียนกลอนทำให้ token โดนซอยและใหญ่ขึ้นมากกว่าเดิมเยอะมาก (~43%)

## The effect on language models

Next, we will see the effect of using "cross-domain" tokenizers on Language models.

### Setup
We are going to reuse the code from the last assignment

In [42]:
# !pip install lightning
sp_pam = sp_pam_uni
sp_pantip = sp_pantip_uni

In [38]:
import itertools
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import lightning as L
from tqdm import tqdm
import numpy as np

In [39]:
class TextDataset(Dataset):
  def __init__(self, data, tokenizer, seq_len = 128):

    token_ids = [tokenizer.encode(d, add_bos=True, add_eos=True) for d in data]
    flatten_token_ids = list(itertools.chain(*token_ids))
    encoded = torch.LongTensor(flatten_token_ids)

    left_over = len(encoded) % seq_len
    encoded = encoded[:len(encoded)-left_over]
    self.encoded = encoded.view(-1, seq_len)

  def __getitem__(self, idx):
    return self.encoded[idx]

  def __len__(self):
    return len(self.encoded)

In [43]:
class LSTM(L.LightningModule):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, learning_rate, criterion):

        super().__init__()

        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.vocab_size=vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.learning_rate = learning_rate
        self.criterion = criterion

    def forward(self, src):
        embedded = self.embedding(src)
        embedded = self.dropout(embedded)
        output, (hidden, cell) = self.lstm(embedded)
        output = self.dropout(output)
        prediction = self.fc(output)
        return prediction

    def training_step(self, batch, batch_idx):

        src = batch[:, :-1]
        target = batch[:, 1:]
        prediction = self(src)
        prediction = prediction.reshape(-1, self.vocab_size)
        target = target.reshape(-1)
        loss = self.criterion(prediction, target)
        self.log("train_loss", loss)
        return loss

    def test_step(self, batch, batch_idx, dataloader_idx=0):

        src = batch[:, :-1]
        target = batch[:, 1:]
        with torch.no_grad():
          prediction = self(src)
        prediction = prediction.reshape(-1, self.vocab_size)
        target = target.reshape(-1)
        loss = self.criterion(prediction, target)
        self.log("test_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

In [44]:
vocab_size = sp_pam.get_piece_size()
embedding_dim = 200
hidden_dim = 512
num_layers = 3
dropout_rate = 0.2
lr = 1e-3
criterion = nn.CrossEntropyLoss()
train_batch_size = 64
test_batch_size = 128

### Training

<a name="no1"></a>
#### 1. Training on Pantip data with Pantip tokenizer

In [45]:
trainer = L.Trainer(
    max_epochs=10,
    deterministic=True
)
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pantip)
pantip_train_loader = DataLoader(pantip_train_dataset, batch_size = train_batch_size, shuffle = True)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pantip)
pantip_test_loader = DataLoader(pantip_test_dataset, batch_size = test_batch_size, shuffle = False)

pam_train_dataset = TextDataset(pam_train_text, sp_pantip)
pam_train_loader = DataLoader(pam_train_dataset, batch_size = train_batch_size, shuffle = True)

pam_test_dataset = TextDataset(pam_test_text, sp_pantip)
pam_test_loader = DataLoader(pam_test_dataset, batch_size = test_batch_size, shuffle = False)

trainer.fit(model, train_dataloaders=pantip_train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/andre/anaconda3/envs/ML/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_fl

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [46]:
test_result = trainer.test(model, dataloaders=[pantip_train_loader, pam_train_loader, pantip_test_loader,pam_test_loader], verbose=False)

print(f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}")
print(f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}")
print(f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}")
print(f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/andre/anaconda3/envs/ML/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/andre/anaconda3/envs/ML/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

Perplexity on Pantip train set is:	61.281180763409324
Perplexity on Pra apai manee train set is:	103.4146114028789
Perplexity on Pantip test set is:	97.12962890710848
Perplexity on Pra apai manee test set is:	104.88839510402421


<a name="no2"></a>
#### 2. Training on Pantip data with Pra apai manee tokenizer

In [47]:
trainer = L.Trainer(
    max_epochs=10,
    deterministic=True
)
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pam)
pantip_train_loader = DataLoader(pantip_train_dataset, batch_size = train_batch_size, shuffle = True)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pam)
pantip_test_loader = DataLoader(pantip_test_dataset, batch_size = test_batch_size, shuffle = False)

pam_train_dataset = TextDataset(pam_train_text, sp_pam)
pam_train_loader = DataLoader(pam_train_dataset, batch_size = train_batch_size, shuffle = True)

pam_test_dataset = TextDataset(pam_test_text, sp_pam)
pam_test_loader = DataLoader(pam_test_dataset, batch_size = test_batch_size, shuffle = False)

trainer.fit(model, train_dataloaders=pantip_train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | embedding | Embedding        | 200 K  | train
1 | lstm      | LSTM             | 5.7 M  | train
2 | dropout   | Dropout          | 0      | train
3 | fc        | Linear           | 513 K  | train
4 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.511    Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [48]:
test_result = trainer.test(model, dataloaders=[pantip_train_loader, pam_train_loader, pantip_test_loader,pam_test_loader], verbose=False)

print(f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}")
print(f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}")
print(f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}")
print(f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

Perplexity on Pantip train set is:	40.536132506580444
Perplexity on Pra apai manee train set is:	452.1203227513103
Perplexity on Pantip test set is:	50.10308220020433
Perplexity on Pra apai manee test set is:	428.1358761299519


#### To answer

The perplexity numbers should indicate that:
1. Training the LM with Pra apai manee tokenizer on Pantip (no. [2](#no2)) results in overfitting to Pantip and poor generalization to the Pra apai manee dataset.
2. However using the Pantip tokenizer (no. [1](#no1)) results in a much better generalization.

Try and come up with some reasons for the results above. <br>
Hint:
1. think about "general" vocabs and domain-specific vocabs.
2. what do you think happens to the model when the token ids become longer.

**ANS.** This happen because from the Q4, tokens tokenized increase almost 13% when tokenize pantip with Pra apai manee tokenizer comparing tp pantip tokenizer. This bigger token lead to less meaning of each token and make the model harder to learn the pattern resulting in overfitting and poor generalization.


<a name="no3"></a>
#### 3. Training on Pra apai manee data with Pantip tokenizer


In [49]:
trainer = L.Trainer(
    max_epochs=10,
    deterministic=True
)
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pantip)
pantip_train_loader = DataLoader(pantip_train_dataset, batch_size = train_batch_size, shuffle = True)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pantip)
pantip_test_loader = DataLoader(pantip_test_dataset, batch_size = test_batch_size, shuffle = False)

pam_train_dataset = TextDataset(pam_train_text, sp_pantip)
pam_train_loader = DataLoader(pam_train_dataset, batch_size = train_batch_size, shuffle = True)

pam_test_dataset = TextDataset(pam_test_text, sp_pantip)
pam_test_loader = DataLoader(pam_test_dataset, batch_size = test_batch_size, shuffle = False)

trainer.fit(model, train_dataloaders=pam_train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | embedding | Embedding        | 200 K  | train
1 | lstm      | LSTM             | 5.7 M  | train
2 | dropout   | Dropout          | 0      | train
3 | fc        | Linear           | 513 K  | train
4 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.511    Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [50]:
test_result = trainer.test(model, dataloaders=[pantip_train_loader, pam_train_loader, pantip_test_loader,pam_test_loader], verbose=False)

print(f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}")
print(f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}")
print(f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}")
print(f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

Perplexity on Pantip train set is:	3524.543551608416
Perplexity on Pra apai manee train set is:	34.48745966133288
Perplexity on Pantip test set is:	3160.6853277452833
Perplexity on Pra apai manee test set is:	37.30071868740598


<a name="no4"></a>
#### 4. Training on Pra apai manee data with Pra apai manee tokenizer




In [51]:
trainer = L.Trainer(
    max_epochs=10,
    deterministic=True
)
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, lr, criterion)

pantip_train_dataset = TextDataset(pantip_train_text, sp_pam)
pantip_train_loader = DataLoader(pantip_train_dataset, batch_size = train_batch_size, shuffle = True)

pantip_test_dataset = TextDataset(pantip_test_text, sp_pam)
pantip_test_loader = DataLoader(pantip_test_dataset, batch_size = test_batch_size, shuffle = False)

pam_train_dataset = TextDataset(pam_train_text, sp_pam)
pam_train_loader = DataLoader(pam_train_dataset, batch_size = train_batch_size, shuffle = True)

pam_test_dataset = TextDataset(pam_test_text, sp_pam)
pam_test_loader = DataLoader(pam_test_dataset, batch_size = test_batch_size, shuffle = False)

trainer.fit(model, train_dataloaders=pam_train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | embedding | Embedding        | 200 K  | train
1 | lstm      | LSTM             | 5.7 M  | train
2 | dropout   | Dropout          | 0      | train
3 | fc        | Linear           | 513 K  | train
4 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.511    Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
/home/andre/anaconda3/envs/ML/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (48) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [52]:
test_result = trainer.test(model, dataloaders=[pantip_train_loader, pam_train_loader, pantip_test_loader,pam_test_loader], verbose=False)

print(f"Perplexity on Pantip train set is:\t{np.exp(test_result[0]['test_loss/dataloader_idx_0'])}")
print(f"Perplexity on Pra apai manee train set is:\t{np.exp(test_result[1]['test_loss/dataloader_idx_1'])}")
print(f"Perplexity on Pantip test set is:\t{np.exp(test_result[2]['test_loss/dataloader_idx_2'])}")
print(f"Perplexity on Pra apai manee test set is:\t{np.exp(test_result[3]['test_loss/dataloader_idx_3'])}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

Perplexity on Pantip train set is:	507.5635309439286
Perplexity on Pra apai manee train set is:	79.43699861301107
Perplexity on Pantip test set is:	495.8720360533206
Perplexity on Pra apai manee test set is:	87.45502393545948


#### To answer

The perplexity numbers should indicate that:
1. Both LM overfits on Pra apai manee data and performs really bad on Pantip data.
2. However using the Pra apai manee tokenizer (no. [4](#no4)) results in a  better generalization than the Pantip tokenizer(no. [3](#no3)).

Try and come up with some reasons for the results above. <br>

As we see from the Q3, general tokenizer perform really worse on sepcial data and cause over 43% more token that general tokenizer (worser that special tokenizer on general data; token bigger ~13%). This lead to loss huge amount of meaning of each token and make the model heavily overfitting. 

And when we guess the special tokenizer on general data, it should be better than general tokenizer on special data from the Q3 and Q4 result. But in this case, the special text is poem lead model to overfitting to poem style and can't generalize to general text.